In [ ]:
!pip install sacrebleu
!pip install datasets
!pip install evaluate
!pip install transformers --upgrade
!pip install peft
!pip install accelerate -U >=0.20.1

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, AutoTokenizer
import evaluate
import numpy as np
from peft import LoraConfig, TaskType, get_peft_model
from huggingface_hub import notebook_login

In [ ]:
sentences = load_dataset('angelacao/gloss_to_spoken5')


In [ ]:

checkpoint = "facebook/bart-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

source_lang = "asl"
target_lang = "en"
prefix = "translate ASL to spoken English: "

In [ ]:
train_df = sentences["train"]
test_df = sentences["test"]
validation_df = sentences["validation"]

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_train = train_df.map(preprocess_function, batched=True)
tokenized_validation = validation_df.map(preprocess_function, batched=True)
tokenized_test = test_df.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

In [ ]:

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

lora_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, r=64, lora_dropout=0.05, lora_alpha=128)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
notebook_login()

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback

training_args = Seq2SeqTrainingArguments(
    output_dir="ASL_to_Spoken",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    push_to_hub=True,
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)],
)

trainer.train()


In [ ]:
results = trainer.evaluate(tokenized_test)

print("Evaluation Results:", results)

In [ ]:

loaded_model = AutoModelForSeq2SeqLM.from_pretrained("angelacao/ASL_to_Spoken")

sample_input = "translate ASL to spoken English: YOU WEDDING WHERE"

sample_input_ids = tokenizer.encode(sample_input, return_tensors="pt")
output_ids = loaded_model.generate(sample_input_ids)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Translated Output:", output_text)